In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
from nltk.corpus import stopwords
from nltk import pos_tag
import os
from textparser import TextParser
import random
from numpy.linalg import norm

In [2]:
random.seed(7)

In [3]:
pd.set_option('max_colwidth', 115)

In [4]:
OHCO = ['movie_id','scene_id', 'sent_num', 'token_num']
SENTS = OHCO[:3]
SCENES = OHCO[:2]
MOVIES = OHCO[:1]

In [5]:
source_files = "/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror"

In [6]:
source_file_list = sorted(glob(f"{source_files}/*.*"))

In [7]:
source_file_list = random.sample(source_file_list, k=30)

In [8]:
movie_titles = []
for source_file_path in source_file_list:
    movie_title = source_file_path.split('/')[-1].split('-')[0].removesuffix('.txt')  
    movie_titles.append((movie_title))

In [9]:
movie_count = list(range(len(source_file_list)))

In [10]:
#year = ['1997', '1998', '1996', '2011', '2009', '2013', '2013', '2009', '2013', '2010', '2014', '2008', '2007', '1999', 
 #      '2014', '1995', '1986', '2009', '2008']

In [11]:
LIB = pd.DataFrame({"movie_id" :movie_count, "source_file_path": source_file_list, "movie_title" : movie_titles})

In [12]:
LIB.set_index('movie_id')

,source_file_path,movie_title
movie_id,,
0,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/jennifersbody.txt,jennifersbody
1,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/finaldestinatio...,finaldestination
2,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/manhunter.txt,manhunter
3,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/they.txt,they
4,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/birdsthe.txt,birdsthe
5,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/case39.txt,case39
6,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/ruinsthe.txt,ruinsthe
7,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/cherryfalls.txt,cherryfalls
8,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/legion.txt,legion


In [13]:
def tokenize_collection(LIB):

    clip_pats = []
    scene_regex = r'^\s*(?:INT\.|EXT\.|INT\/EXT\.|I\/E\/|EST\.|MONTAGE|INT\:|EXT\:|INT|EXT)\s+.*$'

    movies = []
    for movie_id in LIB.index:

        # Announce
        print("Tokenizing", movie_id, LIB.loc[movie_id].movie_title)

        # Define vars
        chap_regex = scene_regex
        ohco_pats = [('scene', chap_regex, 'm')]
        src_file_path = LIB.loc[movie_id].source_file_path

        # Create object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)

        # Define parameters
        text.verbose = False
        text.strip_hyphens = True
        text.strip_whitespace = True

        # Parse
        text.import_source().parse_tokens();

        # Name things
        text.TOKENS['movie_id'] = movie_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['movie_id'] + text.OHCO)

        # Add to list
        movies.append(text.TOKENS)
        
    # Combine into a single dataframe
    CORPUS = pd.concat(movies).sort_index()

    # Clean up
    del(movies)
    del(text)
        
    print("Done")
        
    return CORPUS

CORPUS = tokenize_collection(LIB)

Tokenizing 0 jennifersbody
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 1 finaldestination
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 2 manhunter
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 3 they
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 4 birdsthe
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 5 case39
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 6 ruinsthe
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 7 cherryfalls
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 8 legion
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 9 signs
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 10 bones
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 11 psycho
line_str scene_str
Index(['scene_str'], dtype='object')
Tokenizing 12 gremlins
line_str scene_str
Index(['scene_str']

In [14]:
CORPUS

pos_tuple  pos  token_str  \
movie_id scene_id sent_num token_num                                     
0        1        0        0              (ANITA, NNP)  NNP      ANITA   
                           1            ("NEEDY", NNP)  NNP    "NEEDY"   
                           2          (LESNICKI,, NNP)  NNP  LESNICKI,   
                           3                 (17,, CD)   CD        17,   
                           4               (sits, NNS)  NNS       sits   
...                                                ...  ...        ...   
29       135      10       0                (FADE, NN)   NN       FADE   
                           1                 (TO, NNP)  NNP         TO   
                           2             (BLACK., NNP)  NNP     BLACK.   
                  11       0                 (THE, DT)   DT        THE   
                           1                 (END, NN)   NN        END   

                                      term_str  
movie_id scene_id sent_num token_num            
0        1        0        0             anita  
                           1             needy  
                           2          lesnicki  
                           3                17  
                           4              sits  
...                                        ...  
29       135      10       0              fade  
                           1                to  
                           2             black  
                  11       0               the  
                           1               end  

[684754 rows x 4 columns]

## Extract some features for `LIB`

In [15]:
LIB['movie_len'] = CORPUS.groupby('movie_id').term_str.count()

In [16]:
LIB.head()

,movie_id,source_file_path,movie_title,movie_len
0,0,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/jennifersbody.txt,jennifersbody,22024
1,1,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/finaldestinatio...,finaldestination,23594
2,2,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/manhunter.txt,manhunter,25843
3,3,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/they.txt,they,20910
4,4,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/birdsthe.txt,birdsthe,31328


In [17]:
LIB['n_scenes'] = CORPUS.reset_index()[['movie_id','scene_id']]\
    .drop_duplicates()\
    .groupby('movie_id').scene_id.count()

In [18]:
LIB

,movie_id,source_file_path,movie_title,movie_len,n_scenes
0,0,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/jennifersbody.txt,jennifersbody,22024,101
1,1,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/finaldestinatio...,finaldestination,23594,142
2,2,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/manhunter.txt,manhunter,25843,124
3,3,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/they.txt,they,20910,166
4,4,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/birdsthe.txt,birdsthe,31328,82
5,5,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/case39.txt,case39,22830,160
6,6,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/ruinsthe.txt,ruinsthe,30354,74
7,7,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/cherryfalls.txt,cherryfalls,18734,293
8,8,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/legion.txt,legion,21283,96
9,9,/Users/theothormann/Desktop/Data Science/Spring/DS5001/PossibleCorpus/imsdb_raw_nov_2015/Horror/signs.txt,signs,22191,82


# Exract VOCAB

Extract a vocabulary from the CORPUS as a whole

## Handle Anomalies

NLTK's POS tagger is not perfect -- note the classification of punctuation as nouns, verbs, etc. We remove these from our corups.

In [19]:
CORPUS[CORPUS.term_str == '']

pos_tuple  pos token_str term_str
movie_id scene_id sent_num token_num                                     
1        42       32       9          ("..., NNP)  NNP      "...         
2        1        59       3             (+, NNP)  NNP         +         
         3        4        0             (.., RB)   RB        ..         
         11       0        0              (+, JJ)   JJ         +         
                  7        4             (!", NN)   NN        !"         
...                                           ...  ...       ...      ...
27       47       7        22             (&, CC)   CC         &         
         48       0        5              (&, CC)   CC         &         
         50       61       3              (&, CC)   CC         &         
28       10       20       2               (#, #)    #         #         
                  23       2               (#, #)    #         #         

[194 rows x 4 columns]

In [20]:
CORPUS[CORPUS.term_str == ''].token_str.value_counts()

&             84
+             21
...?          17
..            16
'             12
....           8
"              3
#              2
[              2
]              2
..?            2
!!!            2
/              1
__________     1
'.             1
______         1
**             1
.'             1
.!             1
,,             1
,"             1
"...           1
.)             1
::             1
,.             1
:.             1
_              1
.{             1
",             1
/"             1
?!             1
...!!!!        1
.?             1
!"             1
?...           1
Name: token_str, dtype: int64

In [21]:
CORPUS = CORPUS[CORPUS.term_str != '']

In [22]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['n_pos'] = CORPUS[['term_str','pos']].value_counts().unstack().count(1)

In [23]:
VOCAB

,n,n_chars,p,i,max_pos,n_pos
term_str,,,,,,
0,37,1,0.000054,14.173255,CD,3
000,1,3,0.000001,19.382708,CD,1
0000,1,4,0.000001,19.382708,CD,1
000i12i12,1,9,0.000001,19.382708,CD,1
007,1,3,0.000001,19.382708,CD,1
...,...,...,...,...,...,...
zowie,87,5,0.000127,12.939765,NNP,4
zowies,31,6,0.000045,14.428512,NNP,2
zucchini,1,8,0.000001,19.382708,NN,1


In [24]:
CORPUS

pos_tuple  pos  token_str  \
movie_id scene_id sent_num token_num                                     
0        1        0        0              (ANITA, NNP)  NNP      ANITA   
                           1            ("NEEDY", NNP)  NNP    "NEEDY"   
                           2          (LESNICKI,, NNP)  NNP  LESNICKI,   
                           3                 (17,, CD)   CD        17,   
                           4               (sits, NNS)  NNS       sits   
...                                                ...  ...        ...   
29       135      10       0                (FADE, NN)   NN       FADE   
                           1                 (TO, NNP)  NNP         TO   
                           2             (BLACK., NNP)  NNP     BLACK.   
                  11       0                 (THE, DT)   DT        THE   
                           1                 (END, NN)   NN        END   

                                      term_str  
movie_id scene_id sent_num token_num            
0        1        0        0             anita  
                           1             needy  
                           2          lesnicki  
                           3                17  
                           4              sits  
...                                        ...  
29       135      10       0              fade  
                           1                to  
                           2             black  
                  11       0               the  
                           1               end  

[684560 rows x 4 columns]

## Bring in BOW and TFIDF Functions

### BOW Function

In [25]:
def create_bag(corpus, bag):
    BOW = corpus.groupby(bag+['term_str']).term_str.count().to_frame('n')
    return BOW

### TFIDF Function

In [26]:
def TFIDF(BOW, tf_method):
    BOW_copy = BOW.copy()
    DTCM = BOW.n.unstack()
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log2(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T
    DF = DTCM.count()
    N = DTCM.shape[0]
    IDF = np.log2(N / DF)
    TFIDF = TF * IDF
    DFIDF = DF * IDF
    BOW_copy['tf'] = TF.stack()
    BOW_copy['tfidf'] = TFIDF.stack()
    
    return BOW_copy

## DFIDF Function

In [27]:
def DFIDF(BOW, tf_method):
    BOW_copy = BOW.copy()
    DTCM = BOW.n.unstack()
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log2(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T
    DF = DTCM.count()
    N = DTCM.shape[0]
    IDF = np.log2(N / DF)
    DFIDF = DF * IDF
    
    return DFIDF

## Apply functions to Austen

In [28]:
BOW = create_bag(CORPUS, SCENES)

In [29]:
TFIDF = TFIDF(BOW, 'max')

In [30]:
DFIDF = DFIDF(BOW, 'max')

## Create the Appropriate TFIDF Table

In [31]:
pos_set = ['NN', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']

In [32]:
filtered_VOCAB = VOCAB[VOCAB.max_pos.isin(pos_set)]

In [33]:
filtered_VOCAB

,n,n_chars,p,i,max_pos,n_pos
term_str,,,,,,
03,2,2,0.000003,18.382708,NN,1
05,2,2,0.000003,18.382708,NN,1
08,2,2,0.000003,18.382708,NN,1
0s,6,2,0.000009,16.797746,NN,1
100000,2,6,0.000003,18.382708,NN,1
...,...,...,...,...,...,...
zone,14,4,0.000020,15.575354,NN,2
zoned,2,5,0.000003,18.382708,VBD,2
zoo,3,3,0.000004,17.797746,NN,1


In [34]:
filtered_VOCAB.index

Index(['03', '05', '08', '0s', '100000', '19th', '1os', '2001', '224', '32000',
       ...
       'zipping', 'zips', 'zit', 'zombie', 'zombies', 'zone', 'zoned', 'zoo',
       'zucchini', 'zydeco'],
      dtype='object', name='term_str', length=17699)

In [35]:
TFIDF_unstack = TFIDF.tfidf.unstack(fill_value = 0)

In [36]:
TFIDF_unstack = TFIDF_unstack[filtered_VOCAB.index]

In [37]:
filtered_DFIDF = DFIDF[filtered_VOCAB.index]

In [38]:
TFIDF_main = TFIDF_unstack[filtered_DFIDF.sort_values(ascending=False).head(1000).index]

In [39]:
TFIDF_main

term_str               back       are    looks      then      just      door  \
movie_id scene_id                                                              
0        1         0.105023  0.214208  0.00000  0.000000  0.131680  0.136382   
         2         0.000000  0.123582  0.00000  0.000000  0.151939  0.000000   
         3         0.000000  0.000000  0.00000  0.000000  0.151939  0.000000   
         4         0.000000  0.000000  0.00000  0.000000  0.000000  0.157364   
         5         0.000000  0.000000  0.00000  0.000000  0.658402  0.000000   
...                     ...       ...      ...       ...       ...       ...   
29       131       0.262558  0.000000  0.00000  0.319114  0.329201  0.000000   
         132       0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
         133       0.163673  0.041729  0.11750  0.099464  0.051304  0.265680   
         134       0.000000  0.000000  0.12925  0.000000  0.141086  0.292248   
         135       0.000000  0.000000  0.00000  0.000000  0.151939  0.000000   

term_str                 be       see       not      have  ...  considers  \
movie_id scene_id                                          ...              
0        1         0.273751  0.560366  0.141949  0.000000  ...        0.0   
         2         0.157933  0.161644  0.000000  0.000000  ...        0.0   
         3         0.157933  0.484933  0.163788  0.000000  ...        0.0   
         4         0.157933  0.000000  0.000000  0.000000  ...        0.0   
         5         0.000000  1.400916  0.000000  0.000000  ...        0.0   
...                     ...       ...       ...       ...  ...        ...   
29       131       0.342188  0.350229  0.000000  0.000000  ...        0.0   
         132       0.000000  0.000000  0.000000  0.000000  ...        0.0   
         133       0.000000  0.081872  0.027652  0.027816  ...        0.0   
         134       0.146652  0.000000  0.000000  0.000000  ...        0.0   
         135       0.000000  0.161644  0.000000  0.000000  ...        0.0   

term_str           asked  pants  hurry  womans  instead  ride  diner  eating  \
movie_id scene_id                                                              
0        1           0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
         2           0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
         3           0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
         4           0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
         5           0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
...                  ...    ...    ...     ...      ...   ...    ...     ...   
29       131         0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
         132         0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
         133         0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
         134         0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   
         135         0.0    0.0    0.0     0.0      0.0   0.0    0.0     0.0   

term_str             shoves  
movie_id scene_id            
0        1         0.000000  
         2         0.000000  
         3         0.000000  
         4         0.000000  
         5         0.000000  
...                     ...  
29       131       0.000000  
         132       0.000000  
         133       0.083842  
         134       0.000000  
         135       0.000000  

[4038 rows x 1000 columns]

In [40]:
collapsed = TFIDF_main.groupby(by = 'movie_id').mean()

In [41]:
collapsed

term_str,back,are,looks,then,just,door,be,see,not,have,...,considers,asked,pants,hurry,womans,instead,ride,diner,eating,shoves
movie_id,,,,,,,,,,,,,,,,,,,,,
0,0.057841,0.162729,0.082558,0.052577,0.107902,0.042232,0.094184,0.153865,0.056392,0.088768,...,0.001682,0.000000,0.020714,0.006027,0.000841,0.000000,0.004917,0.000000,0.020355,0.032034
1,0.052863,0.058451,0.074748,0.038735,0.040099,0.078036,0.073637,0.024922,0.086244,0.028349,...,0.041515,0.008246,0.007216,0.000000,0.000000,0.000000,0.012351,0.000000,0.000000,0.000000
2,0.073722,0.136575,0.052462,0.075145,0.026929,0.106242,0.082443,0.121116,0.049180,0.096734,...,0.000000,0.042893,0.012676,0.008677,0.000000,0.009400,0.035219,0.000000,0.002892,0.007448
3,0.063301,0.104119,0.066276,0.046360,0.052763,0.082301,0.051045,0.024222,0.045376,0.042612,...,0.000000,0.000972,0.002992,0.003388,0.011770,0.003536,0.009260,0.000000,0.004321,0.025269
4,0.143241,0.094220,0.124215,0.029658,0.021502,0.067757,0.028412,0.101224,0.021801,0.033239,...,0.000000,0.003643,0.000000,0.003225,0.003028,0.000581,0.000000,0.000000,0.002961,0.002019
5,0.040577,0.041861,0.053768,0.055895,0.072757,0.163770,0.055041,0.074024,0.102670,0.079555,...,0.003104,0.006238,0.000000,0.002242,0.017030,0.000000,0.000000,0.000000,0.039004,0.007111
6,0.099903,0.110432,0.027317,0.219092,0.105199,0.015786,0.055297,0.038147,0.059791,0.094393,...,0.025394,0.002644,0.038596,0.008821,0.002691,0.005965,0.000000,0.004847,0.011184,0.002237
7,0.102042,0.139643,0.086926,0.018834,0.041869,0.129883,0.040369,0.026138,0.043198,0.035796,...,0.000000,0.011257,0.013765,0.011017,0.023411,0.000000,0.002951,0.000000,0.001049,0.003672
8,0.127208,0.055482,0.070020,0.069133,0.103507,0.066729,0.086063,0.069171,0.104969,0.069448,...,0.045151,0.000000,0.000000,0.040349,0.006725,0.004397,0.038797,0.321156,0.000000,0.000000


## Create Normalized Tables

We normalize TFIDF values in various ways. Normalizations types are required for certain distance measures.

In [42]:
L0 = collapsed.astype('bool').astype('int') # Binary (Pseudo L)
L1 = collapsed.apply(lambda x: x / x.sum(), 1) # Probabilistic
L2 = collapsed.apply(lambda x: x / norm(x), 1) # Pythagorean / Euclidean